In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest,chi2

In [6]:
df = pd.read_csv('train.csv')

In [4]:
# df.head()

In [8]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)


In [9]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5,S
733,2,male,23.0,0,0,13.0,S


In [10]:
y_train.head(2)

331    0
733    0
Name: Survived, dtype: int64

In [11]:
# Imputatuon transformer
trf1 = ColumnTransformer([
    ('input_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [13]:
# One-hot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [19]:
# Scaling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [22]:
# Feature Selection(optional)
trf4 = SelectKBest(score_func=chi2,k=8)

In [23]:
# Train the model
trf5 = DecisionTreeClassifier()

In [24]:
# Create Pipeline
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

# Pipeline Vs nake_pipeline 

pipeline require naming of steps, make_pipleine does not.
(Sample applies to Column Transformer vs make_column_transformer)

In [25]:
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [26]:
# Train
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('input_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000026855A88C20>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [27]:
pipe.feature_names_in_

array(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'],
      dtype=object)

In [28]:
pipe.steps

[('columntransformer-1',
  ColumnTransformer(remainder='passthrough',
                    transformers=[('input_age', SimpleImputer(), [2]),
                                  ('impute_embarked',
                                   SimpleImputer(strategy='most_frequent'),
                                   [6])])),
 ('columntransformer-2',
  ColumnTransformer(remainder='passthrough',
                    transformers=[('ohe_sex_embarked',
                                   OneHotEncoder(handle_unknown='ignore',
                                                 sparse_output=False),
                                   [1, 6])])),
 ('columntransformer-3',
  ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))])),
 ('selectkbest',
  SelectKBest(k=8, score_func=<function chi2 at 0x0000026855A88C20>)),
 ('decisiontreeclassifier', DecisionTreeClassifier())]

In [29]:
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('input_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x0000026855A88C20>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [30]:
# Predict
y_pred = pipe.predict(X_test)

In [31]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

In [33]:
# Cross Validation using Pipelines
# cross-validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

np.float64(0.6391214419383433)

# GridSearch using Pipeline

In [34]:
# gridsearchCv
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [37]:
from sklearn.model_selection import GridSearchCV

# Initialize GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')

# Fit on training data
grid.fit(X_train, y_train)


ValueError: Invalid parameter 'trf5' for estimator Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('input_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers

In [36]:
grid.best_score

AttributeError: 'GridSearchCV' object has no attribute 'best_score'

In [ ]:
gridsearch.bestparam

In [ ]:
# Exporitn the pipeline
# import pipeline
# pickle.dump(pipe,open(['pipe.pkl','wb']))


In [ ]:
# # reading the file (.pkl)
# import pickle
# import numpy as np
# pipe = pickle.load('pipe.pkl','rb')
# # Assume user input
# test_input2 = np.array([2,'male',3102,0,0,10.5,'S'],dtype=object).reshape(1,7)
# pipe.predict(test_input2)
